### THỰC HIỆN MÃ HOÁ DES DỮ LIỆU TRUYỀN THÔNG VỚI ĐẦU VÀO ĐẦU RA LÀ FILE VĂN BẢN

#### Giả sử yêu cầu:Cần truyền dữ liệu (1 bài thơ) trong file input.txt,  để bảo mật thì trước khi truyền cần mã hoá DES ,mã hoá xong và lưu vào file transport.txt và truyền đi , đến đích sẽ giải mã DES về dữ liệu gốc, đưa vào file output.txt

 ( Phần code minh hoạ có sử dụng lại một phần code từ Phần tìm hiểu mã hoá khối DES)

In [11]:
# Ma trận hoán vị khởi tạo ban đầu
PI = [58, 50, 42, 34, 26, 18, 10, 2,
      60, 52, 44, 36, 28, 20, 12, 4,
      62, 54, 46, 38, 30, 22, 14, 6,
      64, 56, 48, 40, 32, 24, 16, 8,
      57, 49, 41, 33, 25, 17, 9, 1,
      59, 51, 43, 35, 27, 19, 11, 3,
      61, 53, 45, 37, 29, 21, 13, 5,
      63, 55, 47, 39, 31, 23, 15, 7]

# Ma trận PC_1 hoán vị và nén Key 64 bit -> 56 bit
CP_1 = [57, 49, 41, 33, 25, 17, 9,
        1, 58, 50, 42, 34, 26, 18,
        10, 2, 59, 51, 43, 35, 27,
        19, 11, 3, 60, 52, 44, 36,
        63, 55, 47, 39, 31, 23, 15,
        7, 62, 54, 46, 38, 30, 22,
        14, 6, 61, 53, 45, 37, 29,
        21, 13, 5, 28, 20, 12, 4]

# Ma trận PC_2 hoán vị và nén key 56 bit xuống 48 bit
CP_2 = [14, 17, 11, 24, 1, 5, 3, 28,
        15, 6, 21, 10, 23, 19, 12, 4,
        26, 8, 16, 7, 27, 20, 13, 2,
        41, 52, 31, 37, 47, 55, 30, 40,
        51, 45, 33, 48, 44, 49, 39, 56,
        34, 53, 46, 42, 50, 36, 29, 32]

#Ma trận Expand mở rộng khối Ri từ 32 lên 48 bit
E = [32, 1, 2, 3, 4, 5,
     4, 5, 6, 7, 8, 9,
     8, 9, 10, 11, 12, 13,
     12, 13, 14, 15, 16, 17,
     16, 17, 18, 19, 20, 21,
     20, 21, 22, 23, 24, 25,
     24, 25, 26, 27, 28, 29,
     28, 29, 30, 31, 32, 1]

# Khối S_boxes
S_BOX = [
         
[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
 [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
 [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
 [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],
],

[[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
 [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
 [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
 [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],
],

[[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
 [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
 [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
 [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],
],

[[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
 [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
 [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
 [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],
],  

[[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
 [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
 [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
 [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],
], 

[[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
 [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
 [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
 [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],
], 

[[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
 [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
 [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
 [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],
],
   
[[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
 [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
 [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
 [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],
]
]


#Ma trận P hoán vị 32 bit đầu ra sau khối S-boxes
P = [16, 7, 20, 21, 29, 12, 28, 17,
     1, 15, 23, 26, 5, 18, 31, 10,
     2, 8, 24, 14, 32, 27, 3, 9,
     19, 13, 30, 6, 22, 11, 4, 25]

#Ma trận hoán vị  nghịch đảo với hoán vị khởi tạo IP
PI_1 = [40, 8, 48, 16, 56, 24, 64, 32,
        39, 7, 47, 15, 55, 23, 63, 31,
        38, 6, 46, 14, 54, 22, 62, 30,
        37, 5, 45, 13, 53, 21, 61, 29,
        36, 4, 44, 12, 52, 20, 60, 28,
        35, 3, 43, 11, 51, 19, 59, 27,
        34, 2, 42, 10, 50, 18, 58, 26,
        33, 1, 41, 9, 49, 17, 57, 25]
#Ma trận dịch Key với tương ứng dịch 1 bit tại vòng 1,2,9,16 và 2 bit tại các vòng còn lại
SHIFT = [1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1]

def string_to_bit_array(text):#Hàm đổi văn bản dạng text sang nhị phân
    array = list()
    for char in text:
        binval = binvalue(char, 8)
        array.extend([int(x) for x in list(binval)]) 
    return array

def bit_array_to_string(array): # Hàm đôi nhị phân về dạng text
    res = ''.join([chr(int(y,2)) for y in [''.join([str(x) for x in _bytes]) for _bytes in  nsplit(array,8)]])   
    return res

def binvalue(val, bitsize): # Trả về giá trị nhị phân với 1 chuỗi có kích thức đã cho
    binval = bin(val)[2:] if isinstance(val, int) else bin(ord(val))[2:]
    if len(binval) > bitsize:
        raise "binary value larger than the expected size"
    while len(binval) < bitsize:
        binval = "0"+binval 
    return binval

def nsplit(s, n):#Hàm chia thành các khối có kích thước n
    return [s[k:k+n] for k in range(0, len(s), n)]

ENCRYPT=1    #Mã hoá đặt const =1
DECRYPT=0    # Giải mã đặt const =0

# Class des sẽ chứa các hàm cần thiết để mã hoá DES
class des():
    def __init__(self):
        self.password = None
        self.text = None
        self.keys = list()
        
    def run(self, key, text, action=ENCRYPT, padding=False):
        if len(key) < 8:
            raise "Key Should be 8 bytes long"
        elif len(key) > 8:
            key = key[:8] # Nếu key lớn hơn 8 byte từ sẽ cắt còn 8 byte (64 key)
        
        self.password = key
        self.text = text
        
        if padding and action==ENCRYPT:
            self.addPadding()
        elif len(self.text) % 8 != 0:# Kích thước đã định sẽ là bội số của 8 byte
            raise "Data size should be multiple of 8"
        
        self.generatekeys()# Quá trình tạo key
        text_blocks = nsplit(self.text, 8) #Chia các đoạn text thành các khối 64 bit
        result = list()
        for block in text_blocks:
            block = string_to_bit_array(block)#Chuyển đổi sang dãy bit 
            block = self.permut(block,PI)#Thực hiện hoán vị khởi tạo
            g, d = nsplit(block, 32)#g(LEFT), d(RIGHT)
            tmp = None
            for i in range(16):  #Bắt đầu 16 vòng thuật toán
                d_e = self.expand(d, E) #Mở rộng d (R0) lên 48 bit
                if action == ENCRYPT:
                    tmp = self.xor(self.keys[i], d_e)#Xor với Key(48bit)
                else:
                    tmp = self.xor(self.keys[15-i], d_e)##Bắt đầu giải mã sau kết thúc việc sinh khoá
                tmp = self.substitute(tmp) 
                tmp = self.permut(tmp, P)
                tmp = self.xor(g, tmp)
                g = d
                d = tmp
            result += self.permut(d+g, PI_1) #Thực hiện hoán vị cuối cùng
        final_res = bit_array_to_string(result)
        if padding and action==DECRYPT:
            return self.removePadding(final_res)  #Loại bỏ các bit được padding 
        else:
            return final_res  #Trả về kết quả ciphered/deciphered
    
    def substitute(self, d_e):#Thay thế cac byte dùng Sboxes
        subblocks = nsplit(d_e, 6)
        result = list()
        for i in range(len(subblocks)): 
            block = subblocks[i]
            row = int(str(block[0])+str(block[5]),2)
            column = int(''.join([str(x) for x in block[1:][:-1]]),2)
            val = S_BOX[i][row][column] 
            bin = binvalue(val, 4)
            result += [int(x) for x in bin]
        return result
        
    def permut(self, block, table):#Hoán vị các bit trong khối dùng bảng đã cho
        return [block[x-1] for x in table]
    
    def expand(self, block, table):#Mở rộng các khối dùng bảng cho trước
        return [block[x-1] for x in table]
    
    def xor(self, t1, t2):#Thưc hiện phép XOR 2 khối và trả về kết quả
        return [x^y for x,y in zip(t1,t2)]
    
    def generatekeys(self):#Hàm mô tả lại thuạt toán sinh Key
        self.keys = []
        key = string_to_bit_array(self.password)
        key = self.permut(key, CP_1) #Thực hiện cho key 64 bit qua khối PC_1 thành khối 56 bit
        g, d = nsplit(key, 28) #Chia thành 2 khối 28 bit (g->LEFT),(d->RIGHT)
        for i in range(16):#Ap
            g, d = self.shift(g, d, SHIFT[i]) #Dịch trái xoay vòng
            tmp = g + d 
            self.keys.append(self.permut(tmp, CP_2)) # Qua PC_2 thành Key Ki 48 bit

    def shift(self, g, d, n): #Hàm dịch trái xoay vòng
        return g[n:] + g[:n], d[n:] + d[:n]
    
    def addPadding(self):#Padding bit 0 vào các khối <64 bit
        pad_len = 8 - (len(self.text) % 8)
        self.text += pad_len * chr(pad_len)
    
    def removePadding(self, data):#Loại bỏ các padding thừa khi giải mã xong 
        pad_len = ord(data[-1])
        return data[:-pad_len]
    
    def encrypt(self, key, text, padding=True):
        return self.run(key, text, ENCRYPT, padding)
    
    def decrypt(self, key, text, padding=True):
        return self.run(key, text, DECRYPT, padding)




In [12]:
with open("input.txt", "r+") as file:  # Đọc dữ liệu input
    plaintext =file.read()
    
# Hàm main thực hiện gọi hàm và thực hiện mã hoá giải mã     
if __name__ == '__main__':
    print("Dữ liệu gốc cần truyền đi trong file input.txt:\n \n\n ",plaintext)
    key = "Hoang_N7"                         #KEY ĐẦU VÀO LÀ "Hoang_N7"
    d = des()
    r = d.encrypt(key,plaintext)           # Mã hoá DES dữ liệu đầu vào             
    print("\n\n\n\n\n TIẾN HÀNH MÃ HOÁ DES CHO DỮ LIỆU : \n\n\n\ %r" %r)
    print("\n \n")
       
  

Dữ liệu gốc cần truyền đi trong file input.txt:
 

                  DAY THON VI DA 
                           Tac gia :Han Mac Tu
      Sao anh khong ve choi thon Vi?
      Nhin sang hang cau nang moi len.
      Vuon ai muot qua ,xanh nhu ngoc
      La truc che ngang mat chu dien.
      
      Gio theo loi gio,may duong may.
      Dong nuoc buon thiu ,hoa bap lay...
      Thuyen ai dau ben song trang do,
      Co cho trang ve kip toi nay?
      
      Mo khach duong xa, khach duong xa,
      Ao em trang qua nhin khong ra...
      O day suong khoi mo nhan anh ,
      Ai biet tinh ai co dam da?
      
      





 TIẾN HÀNH MÃ HOÁ DES CHO DỮ LIỆU : 


\ '_\x1b^Ñ\x8d\x1eÛs_\x1b^Ñ\x8d\x1eÛsü\x16ªÀ4â y\x18æ®\xa0\x85\x92C¬_\x1b^Ñ\x8d\x1eÛs_\x1b^Ñ\x8d\x1eÛs_\x1b^Ñ\x8d\x1eÛsäôH\x90`Â··ieùUwbð¥\x06ñu>\xad1\x04\x0bÖca\x0cêe\x90C\x97k\x8fL\x07{v·U\x19r\x87ðW_§-\x82\x13U¸cäPêéß{\x8e\x06³Ñ&û¡Q·\x8f²¾\x12 \x04\x8c¦\x14Coü4/£i}\x9a®Õ\x03$\x8eCqñz\x18@YL×½Ñi¯>ÛÑ\x17uÇÆj\\¦]ÈgÝi4\x07É\x05þÈ±Õ3|\x8a$¬

In [13]:
with open("transport.txt", "w",encoding="utf-8") as file:     # Ghi Cinpher vào file transport.txt
    file.writelines(r)
with open("transport.txt", "r+",encoding="utf-8") as file:    # Đọc dữ liệu trong file transport.txt
    p =file.read()
    print("Dữ liệu được mã hoá và truyền đi bằng file transport.txt: \n \n %r" %p)
    print("\n \n \n")

Dữ liệu được mã hoá và truyền đi bằng file transport.txt: 
 
 '_\x1b^Ñ\x8d\x1eÛs_\x1b^Ñ\x8d\x1eÛsü\x16ªÀ4â y\x18æ®\xa0\x85\x92C¬_\x1b^Ñ\x8d\x1eÛs_\x1b^Ñ\x8d\x1eÛs_\x1b^Ñ\x8d\x1eÛsäôH\x90`Â··ieùUwbð¥\x06ñu>\xad1\x04\x0bÖca\x0cêe\x90C\x97k\x8fL\x07{v·U\x19r\x87ðW_§-\x82\x13U¸cäPêéß{\x8e\x06³Ñ&û¡Q·\x8f²¾\x12 \x04\x8c¦\x14Coü4/£i}\x9a®Õ\x03$\x8eCqñz\x18@YL×½Ñi¯>ÛÑ\x17uÇÆj\\¦]ÈgÝi4\x07É\x05þÈ±Õ3|\x8a$¬Å\x03#¶sÐÒ¤Häö\x93]TÐ£\x01Ò\x19^\t?!ÍÛÀ8àË¿kMH¶<>¤\x82ðs¾-¨Ø&M\x16G\x15\x85\x06A\x98ãZê\x9a¬´Ò\x0fê\x9fÈ§äh\x82)þíd\x9e¶\x14i\x89qúI:wbõð\x859«¯u~¶Ã¢\xa0)`§¿\x87«ùg\x93XDÿ\x07e¥o1eè"µR\x1dÈÀû[ät!¡i/Ì\n\x94~\x1c\x86>é¯l´\x19\x19\x84)ºód:\x7f~\x9f.%Ë+°1\x93\x7fÃN\x0e\x9d(\nk´3\x00èH\x9c\x89ò=÷\x80\x87þìúv\x80Oy\t\x9a\x84æ@o£¢0p\x04ÙÜ\x89tEjÄN\x87QT\x16é,<\x1fe\x7fV]¿ÐkÚó¤Ë?æ\x94\x12¢ºU²#\t©Ã×ó¤Ë?æ\x94\x12¢LÜ:ò\x0bõ\x1bg\x13\x066Cùµªä·C\x1b\x0f"YÛ?0\x02ØÕ\x16\x13¸\x88G-,\x01Å\x98Ø\x14¿DÙ\x97³\x8e\x03ÑÒ¢hzØ~÷í\x8c1\x07ækÇg\x1eß\x1dV]w\t\xa07[§[Tæ\x90¡ê)\x94ÍæÑz%t2*ÿpe,å¤\x0cÔ\nÅ\x94m®\x8cYÿp»ïÖ@ú\

In [14]:
with open("transport.txt", "r+",encoding="utf-8") as file:    # Đọc dữ liệu trong file transport.txt và tiến hành giải mã
    p =file.read()  
    r2 = d.decrypt(key,p)        # Giải mã dữ liệu
    print("Dữ liệu sau giải mã và ghi vào file output.txt : \n\n\n ", r2)
with open("output.txt", "w",encoding="utf-8") as file:     # Ghi dữ liệu sau giải mã  vào file output.txt
    file.writelines(r2)

Dữ liệu sau giải mã và ghi vào file output.txt : 


                  DAY THON VI DA 
                           Tac gia :Han Mac Tu
      Sao anh khong ve choi thon Vi?
      Nhin sang hang cau nang moi len.
      Vuon ai muot qua ,xanh nhu ngoc
      La truc che ngang mat chu dien.
      
      Gio theo loi gio,may duong may.
      Dong nuoc buon thiu ,hoa bap lay...
      Thuyen ai dau ben song trang do,
      Co cho trang ve kip toi nay?
      
      Mo khach duong xa, khach duong xa,
      Ao em trang qua nhin khong ra...
      O day suong khoi mo nhan anh ,
      Ai bie®ø+S$t1i co dam da?
      
      


------Kết thúc------